In [ ]:
import gc
import pickle
import time
import re
from collections import defaultdict
import pandas as pd

from selenium import webdriver  
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException

In [ ]:
driver = webdriver.Chrome("chromedriver/chromedriver.exe")
driver.get('http://kcna.kp/kcna.user.home.retrieveHomeInfoList.kcmsf')

#change language to English
driver.find_element_by_class_name('htop_right_area').find_elements_by_tag_name('a')[1].click()

#navigate to "Latest News"
driver.find_element_by_class_name('htop_menu').find_elements_by_tag_name('li')[1].click()

#get the page number for the last page of articles
driver.find_element_by_class_name('end').click()
total_pages = int(driver.find_element_by_class_name('cur_page').text)
driver.find_element_by_class_name('first').click()

In [ ]:
def load_check(title):
#function to ensure contents are loaded in article detail panel
    attempts = 0
    #loop until article's title matches title in pane or 5 attempts have been made
    while (title != driver.find_element_by_id('article_title').text) and (attempts < 5):
        link.find_element_by_tag_name('a').click()
        time.sleep(2)
        attempts += 1
        continue
    if attempts == 5:
        print("Something went wrong for article '{}'".format(title))
    return

def pickle_dict(dictionary, year):
    with open('kcna_kp_articles_' + year, 'wb') as file:
            pickle.dump(dictionary, file)
    print("Pickled {} articles dictionary".format(year))

def save_check(article_date, article_list):
#check if article is from different year than previous article, if so pickle current dict, create new one
    current_article_year = article_date[:4]
    last_article_year = list(article_list.keys())[-1][:4]
    if int(current_article_year) < int(last_article_year):
        pickle_dict(article_list, last_article_year)
        return(defaultdict(list))
    else:
        return(article_list)

In [ ]:
articles_info = defaultdict(list)

while True:
    current_page = int(driver.find_element_by_class_name('cur_page').text)
    if current_page < total_pages:
        for link in driver.find_element_by_class_name('left_articles').find_elements_by_tag_name('li'):
            article_date = re.findall('\d{4}\.\d{2}\.\d{2}', link.text)[0]
            if len(list(articles_info.keys())) > 0:
                articles_info = save_check(article_date, articles_info)
                gc.collect()
            article_title = link.text.split(" (" + article_date)[0]
            load_check(article_title)
            article_text = driver.find_element_by_class_name('d_a_text').text
            articles_info[article_date].append({"title": article_title, "text": article_text})
    else:
        break
    driver.find_element_by_class_name('next').click()
    #wait for next page's contents to load before continuing
    WebDriverWait(driver, 10).until(EC.staleness_of(link))

In [ ]:
with open('kcna_kp_articles_2013', 'wb') as file:
    pickle.dump(articles_info, file)

In [ ]:
driver.quit()